## Proyecto Final

pip install pandas as pd
pip install numpy as np
pip install pandas pyodbc

In [3]:
# Carga de librerias
import os

## Revisión archivo 2017PurchasePicesDec

In [4]:
from google.cloud import bigquery

# Ruta al archivo JSON de credenciales
key_path = 'elixiemporium-0b9fa2f9c940.json'

# Inicializar el cliente de BigQuery
client = bigquery.Client.from_service_account_json(key_path)

# Especificar la tabla
table_id = 'elixiemporium.elixiremporiumdatossinETL.2017PurchasePricesDec'

# Obtener el número de filas
query_job = client.query(f"SELECT COUNT(*) AS total_filas FROM `{table_id}`")
results = query_job.result()
for row in results:
    total_filas = row.total_filas
    print(f"Total de filas: {total_filas}")

# Obtener el número de columnas (utilizando el esquema de la tabla)
table = client.get_table(table_id)
total_columnas = len(table.schema)
print(f"Total de columnas: {total_columnas}")

Total de filas: 12261
Total de columnas: 9


In [5]:
pip install db-dtypes

In [1]:
from google.cloud import bigquery
import pandas as pd

# Autenticación con el archivo JSON
client = bigquery.Client.from_service_account_json("elixiemporium-0b9fa2f9c940.json")

# Definir el proyecto, dataset y tabla
project_id = "elixiemporium"
dataset_id = "elixiremporiumdatossinETL"
table_id = "2017PurchasePricesDec"
table_full_name = f"{project_id}.{dataset_id}.{table_id}"

# Obtener el esquema de la tabla para listar las columnas
table = client.get_table(table_full_name)
columns = [schema_field.name for schema_field in table.schema]

# Construir la consulta SQL dinámicamente
null_counts_query = "SELECT " + ", ".join(
    [f"SUM(CASE WHEN `{col}` IS NULL THEN 1 ELSE 0 END) AS `{col}_nulls`" for col in columns]
) + f" FROM `{table_full_name}`"

# Ejecutar la consulta
query_job = client.query(null_counts_query)
df = query_job.to_dataframe()

# Transponer el DataFrame para mostrar los valores por columna
df_transposed = df.T.reset_index()
df_transposed.columns = ["Column", "Null_Count"]
df_transposed = df_transposed.sort_values(by="Null_Count", ascending=False)

# Mostrar los resultados
print(df_transposed)

c:\Users\Cami\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                 Column  Null_Count
0           Brand_nulls           0
1     Description_nulls           0
2           Price_nulls           0
3            Size_nulls           0
4          Volume_nulls           0
5  Classification_nulls           0
6   PurchasePrice_nulls           0
7    VendorNumber_nulls           0
8      VendorName_nulls           0


In [3]:
from google.cloud import bigquery
import pandas as pd

# Autenticación con el archivo JSON
client = bigquery.Client.from_service_account_json("elixiemporium-0b9fa2f9c940.json")

# Definir el proyecto, dataset y tabla
project_id = "elixiemporium"
dataset_id = "elixiremporiumdatossinETL"
table_id = "2017PurchasePricesDec"
table_full_name = f"{project_id}.{dataset_id}.{table_id}"

# Obtener el esquema de la tabla para listar las columnas
table = client.get_table(table_full_name)
columns = [schema_field.name for schema_field in table.schema]

# Construir la consulta SQL dinámicamente
null_counts_query = "SELECT " + ", ".join(
    [f"SUM(CASE WHEN `{col}` IS NULL THEN 'NaN' ELSE CAST({col} AS STRING) END) AS `{col}`" for col in columns]
) + f" FROM `{table_full_name}`"

# Ejecutar la consulta
query_job = client.query(null_counts_query)
df = query_job.to_dataframe()

# Transponer el DataFrame (optional)
# This step is optional if you prefer the output in row format
df_transposed = df.T.reset_index()
df_transposed.columns = ["Column", "Value"]
df_transposed = df_transposed.sort_values(by="Column")  # Sort by column name

# Mostrar los resultados
print(df_transposed if len(df_transposed) > 0 else "No data found")

BadRequest: 400 No matching signature for aggregate function SUM
  Argument types: STRING
  Signature: SUM(INT64)
    Argument 1: Unable to coerce type STRING to expected type INT64
  Signature: SUM(FLOAT64)
    Argument 1: Unable to coerce type STRING to expected type FLOAT64
  Signature: SUM(NUMERIC)
    Argument 1: Unable to coerce type STRING to expected type NUMERIC
  Signature: SUM(BIGNUMERIC)
    Argument 1: Unable to coerce type STRING to expected type BIGNUMERIC
  Signature: SUM(INTERVAL)
    Argument 1: Unable to coerce type STRING to expected type INTERVAL at [1:8]; reason: invalidQuery, location: query, message: No matching signature for aggregate function SUM
  Argument types: STRING
  Signature: SUM(INT64)
    Argument 1: Unable to coerce type STRING to expected type INT64
  Signature: SUM(FLOAT64)
    Argument 1: Unable to coerce type STRING to expected type FLOAT64
  Signature: SUM(NUMERIC)
    Argument 1: Unable to coerce type STRING to expected type NUMERIC
  Signatur

In [4]:
from google.cloud import bigquery
import pandas as pd

# Ruta al archivo JSON de credenciales
key_path = 'elixiemporium-0b9fa2f9c940.json'

# Inicializar el cliente de BigQuery
client = bigquery.Client.from_service_account_json(key_path)

# Especificar la tabla
project_id = "elixiemporium"
dataset_id = "elixiremporiumdatossinETL"
table_id = "2017PurchasePricesDec"
table_full_name = f"{project_id}.{dataset_id}.{table_id}"

# Obtener el esquema de la tabla para listar las columnas
table = client.get_table(table_full_name)
columns = [schema_field.name for schema_field in table.schema]

# Construir la consulta SQL dinámicamente
null_counts_query = f"""
SELECT 
    SUM(CASE WHEN {column} IS NULL THEN 1 ELSE 0 END) AS total_nulls
FROM 
    `{table_full_name}`
"""

# Ejecutar la consulta
query_job = client.query(null_counts_query)
results = query_job.result()

# Imprimir los resultados
for row in results:
    print(f"Total de valores nulos: {row.total_nulls}")

NameError: name 'column' is not defined